In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime
import pandas as pd
from string import punctuation
from nltk.tokenize import MWETokenizer
from nltk import ngrams
import itertools
import os

# Function

#### Tìm url theo keyword

In [2]:
def search_product(driver, keyword): # Tìm url sản phẩm theo keyword
    keyword = keyword.lower()

    url = 'https://tiki.vn/search?q=%s'%('%20'.join(keyword.split(' ')))
    driver.get(url)
    sleep(3)

    element= WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="CatalogProducts__Wrapper-sc-1r8ct7c-0 fKlIZg"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup = BeautifulSoup(html_of_interest, 'lxml')

    raw_links = soup.select('a[data-view-id="product_list_item"]')
    list_links = []

    for num_link in range(len(raw_links)):
        dict_links = {}

        # Url của sản phẩm
        link = raw_links[num_link].attrs['href']
        if 'tka.tiki.vn' in link:
            dict_links['url'] = 'https:' + link
        else:
            dict_links['url'] = 'https://tiki.vn' + link
        
        # Tên sản phẩm
        dict_links['Product name'] = soup.select('h3[class="style__NameStyled-sc-2j4n5s-8 juFQqr product-name"]')[num_link].text

        # Brand
        dict_links['Brand'] = soup.select('div[class="style__AboveProductNameStyled-sc-m30gte-0 hjPFIz above-product-name-info"] > span')[num_link].text

        list_links.append(dict_links)

    return list_links

#### Tiền xử lí dữ liệu

In [3]:
def preprocessing(text): # Hàm tiền xử lí dữ liệu string và trả về 1 string
    # Chữ hoa thành chữ thường
    pre_text = text.lower()

    # Loại bỏ dấu câu
    for c in punctuation:
        pre_text= pre_text.replace(c,' ')
    
    pre_text = " ".join(pre_text.split())

    return pre_text

#### Lọc sản phẩm không liên quan

In [4]:
def remove_accessory(list_links): # Hàm loại bỏ sản phẩm phụ kiện (ốp, bao da, kính cường lực) và trả về 1 list
    # Xóa tên thương hiệu trong Tên sản phẩm
    for link in list_links:
        split_link = link['Product name'].split(' ')
        split_test = link['Product name'].lower().split(' ')
        brand = link['Brand'].lower()
        if brand in split_test:
            brand_index = split_test.index(brand)
            split_link.pop(brand_index)
            link['Product name'] = ' '.join(split_link)

    with open(r'..\accessory_keyword.txt', encoding='utf-8') as f:
        accessory_keyword = f.read().splitlines()

    # Loại bỏ từ khóa chứa trong accessory_keyword
    exclude_links = []

    for link in list_links:
        for acc_kw in accessory_keyword:
            if acc_kw in link['Product name'].lower():
                exclude_links.append(link)

    list_links = [link for link in list_links if link not in exclude_links]

    return list_links

In [5]:
def extract_similar_keywords(text, keyword): # Hàm trích xuất từ khóa liên quan đến keyword trong text và trả về 1 list
    smartphone_name = pd.read_csv('..\smartphones.csv')['model']
    similar_keywords_list = [name.lower() for name in smartphone_name if keyword in name.lower()]
    extracted_keyword_list = []
    main_topic_keyword_list = []

    for similar_keyword in similar_keywords_list:
        keyword_ngram_list = []
        for n in range(2, len(similar_keyword)):
            n_gram = ngrams(similar_keyword.split(), n)

            for grams in n_gram:
                keyword_ngram_list.append(list(grams))    

        for keyword_ngram in keyword_ngram_list:
            tokenizer = MWETokenizer()
            tokenizer.add_mwe(keyword_ngram)
            phrase_list = tokenizer.tokenize(text.split())

            topic_keyword = '_'.join(keyword_ngram)
            if topic_keyword in phrase_list:
                extracted_keyword_list.append(keyword_ngram)

    extracted_keyword_list.sort()
    extracted_keyword_list = list(l for l,_ in itertools.groupby(extracted_keyword_list))

    freq_extracted_keyword = {}

    for l,_ in itertools.groupby(extracted_keyword_list):
        kw = ' '.join(l)
        freq_extracted_keyword[kw] = text.count(kw)

    freq_extracted_keyword = sorted(list(freq_extracted_keyword.items()), key = lambda key : len(key[0]), reverse=True)
    freq_extracted_keyword = {ele[0] : ele[1]  for ele in freq_extracted_keyword}

    freq_key_list = list(freq_extracted_keyword.keys())
    check_freq_dict = freq_extracted_keyword.copy()

    for key in freq_key_list:
        req = freq_extracted_keyword[key]

        for check_key in freq_key_list:
            if (key != check_key) and (key in check_key) and (check_freq_dict[check_key] > 0):
                check_freq_dict[key] -= 1

    for key, value in check_freq_dict.items():
        if value > 0:
            main_topic_keyword_list.append(key)
    
    return main_topic_keyword_list

In [6]:
def filter_links(list_links, keyword): # Hàm loại bỏ accessory, trích xuất url của sản phẩm keyword và trả về 1 list
    filter_video_url = []

    # Loại bỏ accessory
    list_links = remove_accessory(list_links)

    for link in list_links:
        product_name = link['Product name']

        pre_text = preprocessing(product_name)

        similar_keywords = extract_similar_keywords(pre_text, keyword)

        if keyword in similar_keywords:
            filter_video_url.append(link['url'])

    return filter_video_url

#### Lấy thông tin và reviews sản phẩm

In [7]:
def get_url(driver, url): # Hàm mở url 
    driver.get(url)

    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'main')))

    total_height = int(driver.execute_script("return document.body.scrollHeight"))

    for i in range(1, total_height, 5):
        driver.execute_script("window.scrollTo(0, {});".format(i))

    check_total_height = int(driver.execute_script("return document.body.scrollHeight"))
    if check_total_height > total_height:
        for i in range(total_height, check_total_height, 5):
            driver.execute_script("window.scrollTo(0, {});".format(i))

In [8]:
def get_soup(driver, url=None): # Hàm lấy code html của url và trả về soup
    if url != None:
        get_url(driver, url)

    element= WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'main > div[class="ContainerRevamp-sc-1hvvgwz-0 dOGdaN"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup = BeautifulSoup(html_of_interest, 'lxml')

    return soup

In [9]:
def get_product_info(soup): # Hàm trả về 1 dict thông tin sản phẩm từ soup
    dict_product_info = {}

    # Tên sản phẩm
    dict_product_info['Name'] = soup.select('h1[class="Title__TitledStyled-sc-c64ni5-0 iXccQY"]')[0].text

    # Thương hiệu
    dict_product_info['Brand'] = soup.select('a[data-view-id="pdp_details_view_brand"]')[0].text

    # Số lượng bán
    sold_count = soup.select('div[data-view-id="pdp_quantity_sold"]')
    if sold_count != []:
        dict_product_info['Quantity'] = soup.select('div[data-view-id="pdp_quantity_sold"]')[0].text.split()[-1]
    else:
        dict_product_info['Quantity'] = 0

    # Check lượt đánh giá
    review_count = soup.select('a[data-view-id="pdp_main_view_review"]')
    if review_count != []:
        # Số lượt đánh giá
        dict_product_info['Reviews count'] = review_count[0].text.strip('()')

        # Đánh giá sao
        dict_product_info['Star rating'] = soup.select('div[class="styles__StyledReview-sc-1onuk2l-1 dRFsZg"] > div')[0].text
    else:
        dict_product_info['Reviews count'] = 0

    # Giá bán
    dict_product_info['Price'] = soup.select('div[class="product-price__current-price"]')[0].text.replace('₫', '')

    # Ngày crawl
    dict_product_info['Crawl date'] = datetime.today().strftime('%Y-%m-%d')

    # Thông số lựa chọn (Màu sắc, dung lượng,...)
    list_option_label = soup.select('div[class="styles__ProductOptionsWrapper-sc-18rzur4-0 jZCObm"] > div')
    if list_option_label != []:
        for option_label in list_option_label:
            list_option = option_label.select('div[class="styles__OptionListWrapper-sc-1pikfxx-2 jgDdBJ"] > div')
            list_all_option = []

            for option in list_option:
                list_all_option.append(option.text)

            dict_product_info[option_label.attrs['data-view-label']] = ', '.join(list_all_option)

    # Tên shop
    dict_product_info['Shop name'] = soup.select('span[class="seller-name"]')[1].text

    # Đánh giá sao shop
    dict_product_info['Shop rating'] = soup.select('div[class="item review"] > div')[0].text

    # Số lượt đánh giá shop
    dict_product_info['Shop rating count'] = soup.select('div[class="item review"] > div')[1].text.strip('()đánh giá')

    # Thông tin chi tiết
    product_info_soup = soup.select('div[class="WidgetTitle__WidgetContainerStyled-sc-12sadap-0 bufoOo"]')[-5]
    product_info = product_info_soup.select('div > div > div > span')

    for i in range(0, len(product_info), 2):
        # product_info[i].text: nhãn thông tin
        # product_info[i+1].text: thông tin
        dict_product_info[product_info[i].text] = product_info[i+1].text

    # Mô tả sản phẩm
    dict_product_info['Describe'] = soup.select('div[class="WidgetTitle__WidgetContainerStyled-sc-12sadap-0 bufoOo"]')[-4].text

    return dict_product_info

In [10]:
def get_review(driver): # Hàm trả về 1 list reviews sản phẩm
    all_reviews = []

    while True:
        # Bấm nút "xem thêm" để hiển thị tất cả nội dung review
        show_more_btn_list = driver.find_elements(By.CSS_SELECTOR, 'span[class="show-more-content"]')
        if show_more_btn_list != []:
            for show_more_btn in show_more_btn_list:
                show_more_btn.click()

        # Lấy soup mới
        soup = get_soup(driver)

        shop_name = soup.select('span[class="seller-name"]')[1].text

        # Lấy thông tin review
        list_review = soup.select('div[class="style__StyledComment-sc-1y8vww-5 dpVjwc review-comment"]')

        for review in list_review:
            dict_review = {}

            # Tên shop
            dict_review['Shop name'] = shop_name

            # Tên
            dict_review['Reviewer name'] = review.select('div[class="review-comment__user-name"]')[0].text

            # Nội dung
            dict_review['Content'] = review.select('div[class="review-comment__content"]')[0].text

            # Đánh giá
            dict_review['Rating'] = review.select('div[class="review-comment__title"]')[0].text

            # Thời gian đánh giá
            dict_review['Rating date'] = review.select('div[class="review-comment__created-date"] > span')[0].text

            # Crawl date
            dict_review['Crawl date'] = datetime.today().strftime('%Y-%m-%d')

            all_reviews.append(dict_review)
        
        # Check có phải page cuối
        try:
            btn_color = driver.find_element(By.CSS_SELECTOR, 'a[class="btn next"] > svg').get_attribute('color')
            if btn_color == '#C4C4CF':
                break
        except:
            break

        # Bấm chuyển page
        driver.find_element(By.CSS_SELECTOR, 'a[class="btn next"]').click()
        sleep(5)

    return all_reviews

#### Lưu file

In [11]:
def save_csv(list_of_dict, folder_path, file_name):
    save_loc = r'%s%s.csv'%(folder_path, file_name)
    exist_file = os.path.exists(save_loc)

    if not exist_file:
        df = pd.DataFrame(list_of_dict)
        df.to_csv(save_loc, index=False)
    else:
        df = pd.read_csv(save_loc)
        df_new = pd.DataFrame(list_of_dict)
        concat_file = pd.concat([df, df_new], ignore_index = True)

        concat_file.to_csv(save_loc, index=False)

# Main

In [22]:
# Mở trình duyệt Google Chrome
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito') # Tab ẩn danh
options.add_argument("--start-maximized") # Full window
options.add_argument('headless') # Không hiển thị chrome
s = Service('../chromedriver.exe')
driver = webdriver.Chrome(service=s, options=options)

In [23]:
# Nhập keyword (tên sản phẩm cần tìm kiếm)
keyword = input('Keyword: ')

# Tìm sản phẩm dựa vào keyword
list_links = search_product(driver, keyword)

# Lọc sản phẩm không liên quan và trích xuất url
filter_url_list = filter_links(list_links, keyword)

In [24]:
product_info = []
product_reviews = []

for product_url in filter_url_list:
    product_soup = get_soup(driver, url=product_url)

    product_info.append(get_product_info(product_soup))
    
    product_reviews = product_reviews + get_review(driver)

driver.close()

#### Save file

In [25]:
folder_path = r"..\..\Data\Tiki\\"

# Lưu file product_info
file_name_info = '%s_info'%('_'.join(keyword.split(' ')))
save_csv(product_info, folder_path, file_name_info)

# Lưu file reviews
# file_name_reviews = '%s_reviews'%('_'.join(keyword.split(' ')))
# save_csv(product_reviews, folder_path, file_name_reviews)